## Standard Statsmodel Forecast Infer

### Import Libraries

The first step is to import the libraries that we will need.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
wallaroo.__version__

'2023.3.0+f196c8f22'

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [4]:
# used for unique connection names

import string
import random

suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

workspace_name = f'multiple-replica-forecast-tutorial-{suffix}'
pipeline_name = 'bikedaypipe'
model_name = 'bikedaymodel'

## Set the Workspace and Pipeline


In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload Model

This model is a simple ARIMA(1,0,1) with no exogenous variables, hard-coded to forecast seven days out.

Note that this package is being specified as a `python` configuration.

In [6]:
model_file_name = 'forecast.py'

bike_day_model = wl.upload_model(model_name, model_file_name, Framework.PYTHON).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

In [7]:
# Set the deployment to allow for additional engines to run
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(1)
                        .memory("1Gi")
                        .build()
                    )

# pipeline = wl.build_pipeline('pipeline')
pipeline.add_model_step(bike_day_model).deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s ........ ok


name,bikedaypipe
created,2023-06-23 14:48:55.218960+00:00
last_updated,2023-06-23 14:48:55.218960+00:00
deployed,True
tags,
versions,61b054a1-6a1d-4894-8977-dd0f476f18fa
steps,bikedaymodel


### Run Inference
Run a test inference to verify the pipeline is operational.

In [8]:
inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [9]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ...................................... ok


name,bikedaypipe
created,2023-06-23 14:48:55.218960+00:00
last_updated,2023-06-23 14:48:55.218960+00:00
deployed,False
tags,
versions,61b054a1-6a1d-4894-8977-dd0f476f18fa
steps,bikedaymodel
